In [1]:
# -*- coding: utf-8 -*-
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
import warnings
from tqdm import tqdm
warnings.filterwarnings('ignore')

C:\Users\Jinbao\anaconda3\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [2]:
train_data = pd.read_csv('data/dataTrain.csv')
test_data = pd.read_csv('data/dataA.csv')
submission = pd.read_csv('data/submit_example_A.csv')
data_nolabel = pd.read_csv('data/dataNoLabel.csv')
print(f'train_data.shape = {train_data.shape}')
print(f'test_data.shape  = {test_data.shape}')
print(f'data_nolabel.shape  = {data_nolabel.shape}')

train_data.shape = (59872, 48)
test_data.shape  = (49858, 47)
data_nolabel.shape  = (39884, 47)


In [3]:
train_data['f47'] = train_data['f1'] * 10 + train_data['f2']
test_data['f47'] = test_data['f1'] * 10 + test_data['f2']

In [4]:
test_data['label'] = -1
data = pd.concat([train_data, test_data])

In [5]:
features = [f'f{idx}' for idx in range(1, 48)]
feature_map = {}
cnt = 0
for feature in features:
    feature_unique = data[feature].unique()
    d = dict(zip(feature_unique, range(1, len(feature_unique) + 1)))
    feature_map[feature] = d

for feature in features:
    train_data[feature] = train_data[feature].map(feature_map[feature])
    test_data[feature] = test_data[feature].map(feature_map[feature])

In [12]:
train = train_data[features][:50000]
label = train_data['label'][:50000]
test = test_data[features]
print("train_data.shape = ", train.shape)
print("train_label.shape = ", label.shape)
print("test_data.shape = ", test.shape)

train_data.shape =  (50000, 47)
train_label.shape =  (50000,)
test_data.shape =  (49858, 47)


In [7]:
import os
import gc
import joblib
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import StratifiedKFold
from sklearn import metrics, preprocessing
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras.models import Model, load_model
from tensorflow.keras import callbacks
from tensorflow.keras import backend as K
from tensorflow.keras import utils
from tensorflow.keras.layers import Dense, Dropout, Activation
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Flatten, Input, Embedding, BatchNormalization
import warnings
warnings.filterwarnings("ignore")

In [8]:
def create_model(features, feature_map): 
    inputs = []
    outputs = []
    for feature in features:
        num = len(feature_map[feature])
        embed_dim = int(min(np.ceil((num)/2), 10))
        inp = layers.Input(shape=(1,))
#         out = layers.Embedding(num + 1, embed_dim, name=feature)(inp)
#         out = layers.Reshape(target_shape=(embed_dim, ))(out)
        out = inp
        inputs.append(inp)
        outputs.append(out)
    
    x = layers.Concatenate()(outputs)
    x = layers.BatchNormalization()(x)
    
    x = layers.Dense(500, activation="relu")(x)
    x = layers.Dropout(0.3)(x)
    x = layers.BatchNormalization()(x)
    
    x = layers.Dense(300, activation="relu")(x)
    x = layers.Dropout(0.3)(x)
    x = layers.BatchNormalization()(x)
    
    y = layers.Dense(1, activation="sigmoid")(x)

    model = Model(inputs=inputs, outputs=y)
    return model

In [9]:
model = create_model(features, feature_map)

In [10]:
test.head()

,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,...,f38,f39,f40,f41,f42,f43,f44,f45,f46,f47
0,2,1,1,1,1,1,2,1,1,2,...,1,1,1,1,1,1,8,44,49,2
1,1,1,1,1,1,1,2,1,1,2,...,1,1,1,1,1,5,13,90,293,1
2,1,1,2,1,1,1,2,1,1,2,...,1,1,1,1,1,1,1,57,72,1
3,1,1,1,1,1,1,2,1,1,2,...,1,1,1,1,1,1,1,11,4,1
4,2,1,3,1,1,1,2,1,1,2,...,1,1,1,1,1,1,1,35,66,2


In [13]:
oof_preds = np.zeros((len(train)))
test_preds = np.zeros((len(test)))

test = [test[col] for col in features]

skf = StratifiedKFold(n_splits=10)
k = 0
for train_index, test_index in skf.split(train, label):
    K.clear_session()
    x_train, x_test = train.iloc[train_index, :], train.iloc[test_index, :]
    y_train, y_test = label.iloc[train_index], label.iloc[test_index]

    
    x_train = [x_train[col] for col in features]
    x_test = [x_test[col] for col in features]
    
    model = create_model(features, feature_map)
    adam = Adam(learning_rate=0.0001, beta_1=0.9, beta_2=0.999, amsgrad=False)
    model.compile(loss='binary_crossentropy', optimizer=adam, metrics=[tf.keras.metrics.AUC()])
    lr = callbacks.ReduceLROnPlateau(monitor='val_auc', factor=0.1,
                                     patience=3, min_lr=1e-6, mode='max', verbose=1)
    es = callbacks.EarlyStopping(monitor='val_auc', min_delta=0.001, patience=5,
                                 verbose=1, mode='max', baseline=None, restore_best_weights=True)
    model.fit(x_train, y_train,
              validation_data=(x_test, y_test),
              verbose=1,
              batch_size=2048,
              callbacks=[lr, es],
              epochs=50
             )
    valid_fold_preds = model.predict(x_test)
    test_fold_preds = model.predict(test)
    oof_preds[test_index] = valid_fold_preds.ravel()
    test_preds += test_fold_preds.ravel()
    print("KFold = %d, auc = %.4f" % (k, roc_auc_score(y_test, valid_fold_preds)))
    k += 1
    K.clear_session()
print("Overall AUC={}".format(roc_auc_score(label.values, oof_preds)))

Epoch 1/50
22/22 [==============================] - 2s 48ms/step - loss: 0.6836 - auc: 0.7320 - val_loss: 0.6341 - val_auc: 0.8146 - lr: 1.0000e-04
Epoch 2/50
22/22 [==============================] - 1s 33ms/step - loss: 0.5874 - auc: 0.8233 - val_loss: 0.5810 - val_auc: 0.8447 - lr: 1.0000e-04
Epoch 3/50
22/22 [==============================] - 1s 33ms/step - loss: 0.5573 - auc: 0.8334 - val_loss: 0.5404 - val_auc: 0.8559 - lr: 1.0000e-04
Epoch 4/50
22/22 [==============================] - 1s 31ms/step - loss: 0.5408 - auc: 0.8387 - val_loss: 0.5075 - val_auc: 0.8628 - lr: 1.0000e-04
Epoch 5/50
22/22 [==============================] - 1s 32ms/step - loss: 0.5272 - auc: 0.8381 - val_loss: 0.4767 - val_auc: 0.8680 - lr: 1.0000e-04
Epoch 6/50
22/22 [==============================] - 1s 36ms/step - loss: 0.5157 - auc: 0.8442 - val_loss: 0.4563 - val_auc: 0.8722 - lr: 1.0000e-04
Epoch 7/50
22/22 [==============================] - 1s 33ms/step - loss: 0.5097 - auc: 0.8443 - val_loss: 0.4427

22/22 [==============================] - 1s 33ms/step - loss: 0.4814 - auc: 0.8470 - val_loss: 0.4527 - val_auc: 0.8818 - lr: 1.0000e-04
Epoch 12/50
22/22 [==============================] - 1s 31ms/step - loss: 0.4737 - auc: 0.8507 - val_loss: 0.4452 - val_auc: 0.8824 - lr: 1.0000e-04
Epoch 13/50
22/22 [==============================] - 1s 32ms/step - loss: 0.4669 - auc: 0.8523 - val_loss: 0.4392 - val_auc: 0.8832 - lr: 1.0000e-04
Epoch 14/50
22/22 [==============================] - 1s 32ms/step - loss: 0.4596 - auc: 0.8529 - val_loss: 0.4339 - val_auc: 0.8835 - lr: 1.0000e-04
Epoch 15/50
22/22 [==============================] - 1s 29ms/step - loss: 0.4582 - auc: 0.8527 - val_loss: 0.4295 - val_auc: 0.8834 - lr: 1.0000e-04
Epoch 16/50
22/22 [==============================] - 1s 33ms/step - loss: 0.4500 - auc: 0.8548 - val_loss: 0.4254 - val_auc: 0.8830 - lr: 1.0000e-04
Epoch 17/50
22/22 [==============================] - ETA: 0s - loss: 0.4452 - auc: 0.8573
Epoch 17: ReduceLROnPlateau 

Epoch 13/50
22/22 [==============================] - 1s 31ms/step - loss: 0.4665 - auc: 0.8552 - val_loss: 0.4271 - val_auc: 0.8775 - lr: 1.0000e-04
Epoch 14/50
22/22 [==============================] - 1s 34ms/step - loss: 0.4589 - auc: 0.8546 - val_loss: 0.4231 - val_auc: 0.8779 - lr: 1.0000e-04
Epoch 15/50
22/22 [==============================] - 1s 31ms/step - loss: 0.4544 - auc: 0.8579 - val_loss: 0.4197 - val_auc: 0.8768 - lr: 1.0000e-04
Epoch 16/50
22/22 [==============================] - 1s 31ms/step - loss: 0.4476 - auc: 0.8576 - val_loss: 0.4144 - val_auc: 0.8792 - lr: 1.0000e-04
Epoch 17/50
22/22 [==============================] - 1s 33ms/step - loss: 0.4461 - auc: 0.8567 - val_loss: 0.4113 - val_auc: 0.8786 - lr: 1.0000e-04
Epoch 18/50
22/22 [==============================] - 1s 32ms/step - loss: 0.4425 - auc: 0.8584 - val_loss: 0.4089 - val_auc: 0.8782 - lr: 1.0000e-04
Epoch 19/50
22/22 [==============================] - 1s 30ms/step - loss: 0.4380 - auc: 0.8590 - val_loss:

22/22 [==============================] - 1s 32ms/step - loss: 0.4589 - auc: 0.8557 - val_loss: 0.4358 - val_auc: 0.8733 - lr: 1.0000e-04
Epoch 15/50
22/22 [==============================] - 1s 33ms/step - loss: 0.4573 - auc: 0.8536 - val_loss: 0.4296 - val_auc: 0.8733 - lr: 1.0000e-04
Epoch 16/50
22/22 [==============================] - 1s 30ms/step - loss: 0.4506 - auc: 0.8568 - val_loss: 0.4258 - val_auc: 0.8735 - lr: 1.0000e-04
Epoch 17/50
22/22 [==============================] - 1s 32ms/step - loss: 0.4466 - auc: 0.8558 - val_loss: 0.4219 - val_auc: 0.8742 - lr: 1.0000e-04
Epoch 18/50
22/22 [==============================] - 1s 32ms/step - loss: 0.4417 - auc: 0.8575 - val_loss: 0.4186 - val_auc: 0.8745 - lr: 1.0000e-04
Epoch 19/50
22/22 [==============================] - 1s 30ms/step - loss: 0.4358 - auc: 0.8593 - val_loss: 0.4147 - val_auc: 0.8749 - lr: 1.0000e-04
Epoch 20/50
22/22 [==============================] - 1s 32ms/step - loss: 0.4344 - auc: 0.8598 - val_loss: 0.4124 - va

Epoch 22: early stopping
1559/1559 [==============================] - 2s 2ms/step
KFold = 8, auc = 0.8888
Epoch 1/50
22/22 [==============================] - 2s 52ms/step - loss: 0.7458 - auc: 0.6634 - val_loss: 0.7845 - val_auc: 0.7930 - lr: 1.0000e-04
Epoch 2/50
22/22 [==============================] - 1s 33ms/step - loss: 0.6076 - auc: 0.8155 - val_loss: 0.7003 - val_auc: 0.8401 - lr: 1.0000e-04
Epoch 3/50
22/22 [==============================] - 1s 31ms/step - loss: 0.5711 - auc: 0.8265 - val_loss: 0.6985 - val_auc: 0.8520 - lr: 1.0000e-04
Epoch 4/50
22/22 [==============================] - 1s 34ms/step - loss: 0.5484 - auc: 0.8324 - val_loss: 0.6586 - val_auc: 0.8607 - lr: 1.0000e-04
Epoch 5/50
22/22 [==============================] - 1s 34ms/step - loss: 0.5357 - auc: 0.8354 - val_loss: 0.6245 - val_auc: 0.8658 - lr: 1.0000e-04
Epoch 6/50
22/22 [==============================] - 1s 34ms/step - loss: 0.5201 - auc: 0.8382 - val_loss: 0.5898 - val_auc: 0.8695 - lr: 1.0000e-04
Epoch 

In [12]:
print("Overall AUC={}".format(roc_auc_score(label.values, oof_preds)))

Overall AUC=0.770485921874636
